In [74]:
# import pandas as pd
import modin as pd
import boto3
# import pandas as pd
import s3fs
import re
import logging

from botocore.exceptions import ClientError


In [2]:
post_path = "../Data/wallstreetbets_posts.csv"
comment_path = "../Data/wallstreetbets_comments.csv"

In [3]:
comment_df = pd.read_csv(comment_path)
post_df = pd.read_csv(post_path)

In [4]:
comment_df.columns.to_list()

['all_awardings',
 'associated_award',
 'author',
 'author_flair_background_color',
 'author_flair_css_class',
 'author_flair_richtext',
 'author_flair_template_id',
 'author_flair_text',
 'author_flair_text_color',
 'author_flair_type',
 'author_fullname',
 'author_patreon_flair',
 'author_premium',
 'awarders',
 'body',
 'collapsed_because_crowd_control',
 'comment_type',
 'created_utc',
 'gildings',
 'id',
 'is_submitter',
 'link_id',
 'locked',
 'no_follow',
 'parent_id',
 'permalink',
 'retrieved_on',
 'score',
 'send_replies',
 'stickied',
 'subreddit',
 'subreddit_id',
 'top_awarded_type',
 'total_awards_received',
 'treatment_tags',
 'distinguished',
 'author_cakeday']

In [6]:
post_df.columns.to_list()

['all_awardings',
 'allow_live_comments',
 'author',
 'author_flair_background_color',
 'author_flair_css_class',
 'author_flair_richtext',
 'author_flair_text',
 'author_flair_text_color',
 'author_flair_type',
 'author_fullname',
 'author_patreon_flair',
 'author_premium',
 'awarders',
 'can_mod_post',
 'contest_mode',
 'created_utc',
 'domain',
 'full_link',
 'gildings',
 'id',
 'is_crosspostable',
 'is_meta',
 'is_original_content',
 'is_reddit_media_domain',
 'is_robot_indexable',
 'is_self',
 'is_video',
 'link_flair_background_color',
 'link_flair_css_class',
 'link_flair_richtext',
 'link_flair_template_id',
 'link_flair_text',
 'link_flair_text_color',
 'link_flair_type',
 'locked',
 'media_only',
 'no_follow',
 'num_comments',
 'num_crossposts',
 'over_18',
 'parent_whitelist_status',
 'permalink',
 'pinned',
 'pwls',
 'retrieved_on',
 'score',
 'selftext',
 'send_replies',
 'spoiler',
 'stickied',
 'subreddit',
 'subreddit_id',
 'subreddit_subscribers',
 'subreddit_type',
 '

In [11]:
comment_df.isnull().sum()

all_awardings                            0
associated_award                   9559657
author                                   0
author_flair_background_color      9392288
author_flair_css_class             9554484
author_flair_richtext              2357655
author_flair_template_id           9392288
author_flair_text                  8581524
author_flair_text_color            6223708
author_flair_type                  2357655
author_fullname                    2357655
author_patreon_flair               2357655
author_premium                     2357655
awarders                                 0
body                                    10
collapsed_because_crowd_control    9559657
comment_type                       9559657
created_utc                              0
gildings                                 0
id                                       0
is_submitter                             0
link_id                                  0
locked                                   0
no_follow  

In [22]:
len(comment_df)

9559657

In [27]:
comment_df["author"].sample(50)

2929015                  Howdar
297157                 phil6298
2310328               [deleted]
7176026           BeautifulBurd
7353072          DOODJLIGHTNING
7837273               [deleted]
2408248                   takjp
4311419               [deleted]
4479861               [deleted]
7607508             Swellyrides
4368776              redstag191
1918326               [deleted]
7822045         Stonk_Collector
9071726                  peetak
9359739            FLUFFYFRUITS
3849024    undergraduateproject
6766954           AutoModerator
9498849               [deleted]
7740243                 M1chaeI
2813179       existential_plant
5745327         WastedKnowledge
145852                [deleted]
7031781               [deleted]
5299413              Hicklarity
6999057          I_StoleYourCar
5067032               [deleted]
4391079       Italianplayground
1721605               [deleted]
8840976         WagieInTheCagie
7734507               CarRamRob
8010011                Zdubzz19
8559188 

In [32]:
comment_df["author"].value_counts()

[deleted]             2357655
AutoModerator           72804
PlaneIntroduction8      10237
OptionsAndTren           6576
WSBVoteBot               6397
                       ...   
ChefT1982                   1
dustofdeath                 1
Hoghead1000                 1
keatoncollins100            1
Oxyoverrimjobs              1
Name: author, Length: 657292, dtype: int64

In [36]:
cleaned_df = comment_df[comment_df["author"] != "[deleted]"]

In [37]:
cleaned_df

,all_awardings,associated_award,author,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,author_flair_text_color,author_flair_type,...,score,send_replies,stickied,subreddit,subreddit_id,top_awarded_type,total_awards_received,treatment_tags,distinguished,author_cakeday
0,[],NaN,guesswhatihate,NaN,NaN,[],NaN,NaN,NaN,text,...,1,True,False,wallstreetbets,t5_2th52,NaN,0,[],NaN,NaN
1,[],NaN,k12nmonky,NaN,NaN,[],NaN,NaN,NaN,text,...,1,True,False,wallstreetbets,t5_2th52,NaN,0,[],NaN,NaN
2,[],NaN,tnmtnmtnm,NaN,NaN,[],NaN,NaN,NaN,text,...,1,True,False,wallstreetbets,t5_2th52,NaN,0,[],NaN,NaN
3,[],NaN,everlastingdeath,NaN,NaN,[],NaN,NaN,NaN,text,...,1,True,False,wallstreetbets,t5_2th52,NaN,0,[],NaN,NaN
4,[],NaN,ZakkuTurner,NaN,NaN,[],NaN,NaN,NaN,text,...,1,True,False,wallstreetbets,t5_2th52,NaN,0,[],NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9559652,[],NaN,AngryBobRoss,NaN,NaN,[],NaN,NaN,NaN,text,...,1,True,False,wallstreetbets,t5_2th52,NaN,0,[],NaN,NaN
9559653,[],NaN,Machin_Shins,NaN,NaN,[],NaN,NaN,NaN,text,...,2,True,False,wallstreetbets,t5_2th52,NaN,0,[],NaN,NaN
9559654,[],NaN,testfire10,NaN,NaN,[],NaN,NaN,NaN,text,...,22,True,False,wallstreetbets,t5_2th52,NaN,0,[],NaN,NaN
9559655,[],NaN,stupidlysweet,NaN,NaN,[],NaN,NaN,NaN,text,...,12,True,False,wallstreetbets,t5_2th52,NaN,0,[],NaN,NaN


In [54]:
cleaned_df = cleaned_df.drop([ 'author_flair_background_color',
 'author_flair_css_class',
 'author_flair_richtext',
 'author_flair_template_id',
 'author_flair_text',
 'author_flair_text_color',
 'author_flair_type',
 'author_fullname',
 'author_patreon_flair',
 'author_premium',
 'collapsed_because_crowd_control',
 'comment_type', 
 'locked',
 'no_follow', 
 'all_awardings',
 'associated_award', 
 'send_replies', 
 'stickied', 
 'subreddit', 
 'subreddit_id', 
 'distinguished', 
 'author_cakeday'],1)

In [60]:
cleaned_df

,author,awarders,body,created_utc,gildings,id,is_submitter,link_id,parent_id,permalink,retrieved_on,score,top_awarded_type,total_awards_received,treatment_tags
0,guesswhatihate,[],#DIPPING,1607957482,{},gftdfol,False,t3_kcvkwx,t1_gftdcb8,/r/wallstreetbets/comments/kcvkwx/daily_discus...,1607957494,1,NaN,0,[]
1,k12nmonky,[],NIO and SPCE ftw,1607957482,{},gftdfo4,False,t3_kcvkwx,t1_gftdc1s,/r/wallstreetbets/comments/kcvkwx/daily_discus...,1607957494,1,NaN,0,[]
2,tnmtnmtnm,[],Flat,1607957482,{},gftdfo3,False,t3_kcvkwx,t1_gftdbth,/r/wallstreetbets/comments/kcvkwx/daily_discus...,1607957494,1,NaN,0,[]
3,everlastingdeath,[],Lol what happened to the limit up everyone's b...,1607957481,{},gftdfnp,False,t3_kcvkwx,t3_kcvkwx,/r/wallstreetbets/comments/kcvkwx/daily_discus...,1607957494,1,NaN,0,[]
4,ZakkuTurner,[],"Jan 22 2021, 40.5",1607957481,{},gftdfmf,False,t3_kcvkwx,t1_gftddng,/r/wallstreetbets/comments/kcvkwx/daily_discus...,1607957493,1,NaN,0,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9559652,AngryBobRoss,[],Lmao I got 2 nights to sleep over it. I might ...,1611412578,{},gkdmw91,False,t3_l2shj5,t1_gkceomw,/r/wallstreetbets/comments/l2shj5/doing_my_par...,1611571732,1,NaN,0,[]
9559653,Machin_Shins,[],Squeeze aside. Are there price targets for GME...,1611412578,{},gkdmw6k,False,t3_l3c0hj,t3_l3c0hj,/r/wallstreetbets/comments/l3c0hj/considering_...,1611571731,2,NaN,0,[]
9559654,testfire10,[],TD Ameritrade,1611412576,{},gkdmw1y,False,t3_l3cp2s,t1_gkdloa8,/r/wallstreetbets/comments/l3cp2s/gme_gainz_so...,1611571730,22,NaN,0,[]
9559655,stupidlysweet,[],Same. I've got 30k in savings but thinking of ...,1611412575,{},gkdmvxa,False,t3_l35s4w,t1_gkbyl29,/r/wallstreetbets/comments/l35s4w/do_not_sell_...,1611571728,12,NaN,0,[]


In [69]:
cleaned_df = cleaned_df.drop(['top_awarded_type','retrieved_on'],1)

In [70]:
cleaned_df

,author,body,created_utc,id,link_id,parent_id,permalink,score,total_awards_received
0,guesswhatihate,#DIPPING,1607957482,gftdfol,t3_kcvkwx,t1_gftdcb8,/r/wallstreetbets/comments/kcvkwx/daily_discus...,1,0
1,k12nmonky,NIO and SPCE ftw,1607957482,gftdfo4,t3_kcvkwx,t1_gftdc1s,/r/wallstreetbets/comments/kcvkwx/daily_discus...,1,0
2,tnmtnmtnm,Flat,1607957482,gftdfo3,t3_kcvkwx,t1_gftdbth,/r/wallstreetbets/comments/kcvkwx/daily_discus...,1,0
3,everlastingdeath,Lol what happened to the limit up everyone's b...,1607957481,gftdfnp,t3_kcvkwx,t3_kcvkwx,/r/wallstreetbets/comments/kcvkwx/daily_discus...,1,0
4,ZakkuTurner,"Jan 22 2021, 40.5",1607957481,gftdfmf,t3_kcvkwx,t1_gftddng,/r/wallstreetbets/comments/kcvkwx/daily_discus...,1,0
...,...,...,...,...,...,...,...,...,...
9559652,AngryBobRoss,Lmao I got 2 nights to sleep over it. I might ...,1611412578,gkdmw91,t3_l2shj5,t1_gkceomw,/r/wallstreetbets/comments/l2shj5/doing_my_par...,1,0
9559653,Machin_Shins,Squeeze aside. Are there price targets for GME...,1611412578,gkdmw6k,t3_l3c0hj,t3_l3c0hj,/r/wallstreetbets/comments/l3c0hj/considering_...,2,0
9559654,testfire10,TD Ameritrade,1611412576,gkdmw1y,t3_l3cp2s,t1_gkdloa8,/r/wallstreetbets/comments/l3cp2s/gme_gainz_so...,22,0
9559655,stupidlysweet,Same. I've got 30k in savings but thinking of ...,1611412575,gkdmvxa,t3_l35s4w,t1_gkbyl29,/r/wallstreetbets/comments/l35s4w/do_not_sell_...,12,0


In [75]:
cleaned_df.to_csv("cleaned_wsb_comments.csv", index=False)

In [72]:
client = boto3.client('s3')

In [73]:
s3 = boto3.resource('s3')
my_bucket = s3.Bucket('wsb-final-project-bucket')
for my_bucket_object in my_bucket.objects.all():
    print(my_bucket_object)

s3.ObjectSummary(bucket_name='wsb-final-project-bucket', key='aws-programmatic-access-test-object')
s3.ObjectSummary(bucket_name='wsb-final-project-bucket', key='kaggle_wsb_comments_12:6-2:6.csv')
s3.ObjectSummary(bucket_name='wsb-final-project-bucket', key='kaggle_wsb_posts_12:6-2:6.csv')
s3.ObjectSummary(bucket_name='wsb-final-project-bucket', key='test/test/20210101-20210201/b1a0304a-737f-4cc0-bff8-a7911ddf79ba/test-1.csv.gz')
s3.ObjectSummary(bucket_name='wsb-final-project-bucket', key='test/test/20210101-20210201/b1a0304a-737f-4cc0-bff8-a7911ddf79ba/test-Manifest.json')
s3.ObjectSummary(bucket_name='wsb-final-project-bucket', key='test/test/20210101-20210201/test-Manifest.json')
s3.ObjectSummary(bucket_name='wsb-final-project-bucket', key='test/test/20210201-20210301/305336a5-a905-4bf1-8b81-2f5d18953f59/test-1.csv.gz')
s3.ObjectSummary(bucket_name='wsb-final-project-bucket', key='test/test/20210201-20210301/305336a5-a905-4bf1-8b81-2f5d18953f59/test-Manifest.json')
s3.ObjectSummary

In [77]:
response = client.upload_file("../Data/cleaned_wsb_comments.csv", "wsb-final-project-bucket", "cleaned_wsb_comments.csv")

KeyboardInterrupt: 